# Time Series Prediction with Reservoir Computing ($RCN$)

AUTHOR: Anurag Dutta (anuragdutta.research@gmail.com)

SUPERVISOR: Tanujit Chakraborty (tanujit.chakraborty@sorbonne.ae)

CREATED: 2023-03-07 19:59:16

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Gathering Dependencies

_Importing Required Libraries_

In [2]:
pip install hampel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import LSTM
import keras
import tensorflow as tf
from hampel import hampel
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Loading Datasets

_EL NINO_

In [21]:
data = pd.read_csv("/content/drive/MyDrive/new/datasets/elnino.csv")
training_set = data.iloc[:, 1]
print(training_set)

0       23.4
1       23.4
2       24.2
3       24.4
4       25.1
        ... 
1629    25.4
1630    24.9
1631    25.3
1632    24.7
1633    24.6
Name: SST1, Length: 1634, dtype: float64


## Preprocessing the Data

In [22]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(training_set, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[23.4 23.4 24.2] 24.4
[23.4 24.2 24.4] 25.1
[24.2 24.4 25.1] 25.8
[24.4 25.1 25.8] 25.9
[25.1 25.8 25.9] 26.1
[25.8 25.9 26.1] 26.1
[25.9 26.1 26.1] 26.7
[26.1 26.1 26.7] 26.1
[26.1 26.7 26.1] 26.1
[26.7 26.1 26.1] 25.7
[26.1 26.1 25.7] 25.6
[26.1 25.7 25.6] 25.1
[25.7 25.6 25.1] 25.3
[25.6 25.1 25.3] 25.1
[25.1 25.3 25.1] 24.6
[25.3 25.1 24.6] 24.2
[25.1 24.6 24.2] 24.3
[24.6 24.2 24.3] 23.7
[24.2 24.3 23.7] 23.4
[24.3 23.7 23.4] 23.2
[23.7 23.4 23.2] 22.8
[23.4 23.2 22.8] 22.5
[23.2 22.8 22.5] 22.1
[22.8 22.5 22.1] 21.7
[22.5 22.1 21.7] 21.3
[22.1 21.7 21.3] 21.0
[21.7 21.3 21. ] 20.2
[21.3 21.  20.2] 20.6
[21.  20.2 20.6] 20.4
[20.2 20.6 20.4] 20.4
[20.6 20.4 20.4] 19.8
[20.4 20.4 19.8] 20.2
[20.4 19.8 20.2] 19.7
[19.8 20.2 19.7] 20.1
[20.2 19.7 20.1] 20.3
[19.7 20.1 20.3] 20.3
[20.1 20.3 20.3] 20.8
[20.3 20.3 20.8] 20.1
[20.3 20.8 20.1] 20.3
[20.8 20.1 20.3] 20.1
[20.1 20.3 20.1] 20.2
[20.3 20.1 20.2] 20.5
[20.1 20.2 20.5] 20.8
[20.2 20.5 20.8] 20.9
[20.5 20.8 20.9] 21.5
[20.8 20.9

## Model ($RCN$)

In [23]:
pip install reservoirpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
from reservoirpy.nodes import Reservoir, Ridge

reservoir = Reservoir(units=100, lr=0.5, sr=0.001)
readout = Ridge(output_dim=3, ridge=1e-8)

In [26]:
esn = reservoir >> readout

In [27]:
esn.fit(X[:1304], X[1:1305], warmup=100)

Running Model-2:   0%|          | 0/1 [00:00<?, ?it/s]
Running Model-2: 165it [00:00, 1643.26it/s]           
Running Model-2: 334it [00:00, 1666.93it/s]
Running Model-2: 501it [00:00, 1429.85it/s]
Running Model-2: 675it [00:00, 1540.09it/s]
Running Model-2: 833it [00:00, 1137.76it/s]
Running Model-2: 1011it [00:00, 1304.26it/s]
Running Model-2: 1156it [00:00, 1237.92it/s]
Running Model-2: 1304it [00:01, 1201.01it/s]
Running Model-2: 100%|██████████| 1/1 [00:01<00:00,  1.12s/it]


Fitting node Ridge-3...


'Model-2': Model('Reservoir-3', 'Ridge-3')

In [28]:
predictions = esn.run(X[1305:-1])

Running Model-2: 325it [00:00, 3884.79it/s]           


## Metrics ($RCN$)

In [29]:
import reservoirpy.observables
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = np.sqrt(mean_squared_error(X[1305:-1], predictions))
mae = np.sqrt(mean_absolute_error(X[1305:-1], predictions))
print('Test RMSE: %.3f' % rmse)
print('Test MAE: %.3f' % mae)

Test RMSE: 2.481
Test MAE: 1.461
